In [1]:
import MyModel
import pandas as pd
import cv2
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('flora.csv')
df['filename'] = 'cc78ftcdf9-1/' + df['ID'].astype(str).str.zfill(4) + '.jpg'
df

,Unnamed: 0,SPECIES,ID,SET,filename
0,297,Manilkara elata,3135,train,cc78ftcdf9-1/3135.jpg
1,137,Mezilaurus itauba,3307,train,cc78ftcdf9-1/3307.jpg
2,1138,Araucaria angustifolia,223,train,cc78ftcdf9-1/0223.jpg
3,1333,Swietenia macrophylla,4509,train,cc78ftcdf9-1/4509.jpg
4,186,Machaerium paraguariense,2929,train,cc78ftcdf9-1/2929.jpg
...,...,...,...,...,...
1896,905,Peltogyne sp,4116,validation,cc78ftcdf9-1/4116.jpg
1897,1791,Hymenaea sp,2413,validation,cc78ftcdf9-1/2413.jpg
1898,1096,Araucaria angustifolia,205,validation,cc78ftcdf9-1/0205.jpg
1899,235,Dipteryx sp,1606,validation,cc78ftcdf9-1/1606.jpg


In [20]:
# test Le's model
le = MyModel.MyModel()

valid = df.loc[df['SET']=='validation']
for idx in range(10):
    print(df['filename'].values[idx])
    im = cv2.imread(df['filename'].values[idx])
    #plt.imshow(im)
    #plt.show()
    print(le.predict(im))

/home/keith/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


cc78ftcdf9-1/3135.jpg
[30]
cc78ftcdf9-1/3307.jpg
[32]
cc78ftcdf9-1/0223.jpg
[1]
cc78ftcdf9-1/4509.jpg
[44]
cc78ftcdf9-1/2929.jpg
[28]
cc78ftcdf9-1/3515.jpg
[34]
cc78ftcdf9-1/2718.jpg
[26]
cc78ftcdf9-1/1821.jpg
[34]
cc78ftcdf9-1/3924.jpg
[38]
cc78ftcdf9-1/1408.jpg
[13]


In [13]:
# train lr stacked model

from sklearn.linear_model import LogisticRegression

le = MyModel.MyModel()

df = pd.read_csv('flora.csv')
df['filename'] = 'cc78ftcdf9-1/' + df['ID'].astype(str).str.zfill(4) + '.jpg'

# taking class id minus one, following le's prediction output
df['y'] = df['ID'].astype(str).str.zfill(4).str[:2].astype(int)-1 

import random
df['random'] = [random.randrange(0, 46) for iter in range(df.shape[0])]

predictions = []
for idx in range(df.shape[0]):
    #print(test['filename'].values[idx])
    im = cv2.imread(df['filename'].values[idx])
    predictions.append(le.predict(im)[0])

df['le'] = predictions

test = df.loc[df['SET']=='test']
    
#clf = LogisticRegression(random_state=0).fit(X, y)

/home/keith/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [14]:
df.head(5)

,Unnamed: 0,SPECIES,ID,SET,filename,y,random,le
0,297,Manilkara elata,3135,train,cc78ftcdf9-1/3135.jpg,30,1,30
1,137,Mezilaurus itauba,3307,train,cc78ftcdf9-1/3307.jpg,32,1,32
2,1138,Araucaria angustifolia,223,train,cc78ftcdf9-1/0223.jpg,1,1,1
3,1333,Swietenia macrophylla,4509,train,cc78ftcdf9-1/4509.jpg,44,1,44
4,186,Machaerium paraguariense,2929,train,cc78ftcdf9-1/2929.jpg,28,1,28


In [35]:
from sklearn.ensemble import RandomForestClassifier
lr = LogisticRegression(random_state=0, solver = 'lbfgs').fit(test[['le','random']], test['y'])
rf = RandomForestClassifier(random_state=0).fit(test[['le','random']], test['y'])

/home/keith/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
df['lr'] = lr.predict(df[['le','random']])
df['rf'] = rf.predict(df[['le','random']])

In [38]:
#sum(df.loc[df['SET']=='validation']['y'] == df.loc[df['SET']=='validation']['stacked'])
df.head(20)

,Unnamed: 0,SPECIES,ID,SET,filename,y,random,le,stacked,lr,rf
0,297,Manilkara elata,3135,train,cc78ftcdf9-1/3135.jpg,30,12,30,4,4,30
1,137,Mezilaurus itauba,3307,train,cc78ftcdf9-1/3307.jpg,32,41,32,0,0,32
2,1138,Araucaria angustifolia,223,train,cc78ftcdf9-1/0223.jpg,1,13,1,0,0,1
3,1333,Swietenia macrophylla,4509,train,cc78ftcdf9-1/4509.jpg,44,15,44,4,4,44
4,186,Machaerium paraguariense,2929,train,cc78ftcdf9-1/2929.jpg,28,12,28,4,4,28
5,1407,Mimosa scabrella,3515,train,cc78ftcdf9-1/3515.jpg,34,41,34,0,0,34
6,820,Inga vera,2718,train,cc78ftcdf9-1/2718.jpg,26,30,26,0,0,26
7,623,Eucalyptus sp,1821,train,cc78ftcdf9-1/1821.jpg,17,31,34,4,4,34
8,1501,Ocotea indecora,3924,train,cc78ftcdf9-1/3924.jpg,38,15,38,4,4,38
9,1436,Cordia goeldiana,1408,train,cc78ftcdf9-1/1408.jpg,13,39,13,0,0,13
